### ![](../../img/installation-ico.png) Install

In [ ]:
pip install langchain langchain-community langchain-google-vertexai google.auth

Fonctionnalités nécessaires à l'exploitation de fichiers PDF

In [ ]:
pip install pypdf

ChromaDB pour LangChain

In [ ]:
pip install langchain-chroma

### ![](../../img/package-ico.png) Imports

In [ ]:
import os
import google.auth

from langchain_google_vertexai import VertexAI

# pour la récupération de données depuis un fichier PDF
from langchain_community.document_loaders import PyPDFLoader

# pour redécouper les documents
from langchain.text_splitter import RecursiveCharacterTextSplitter

# ChromaDB Vectore Store
from langchain.vectorstores import Chroma

# modèle d'embedding de documents
from langchain_core.embeddings import FakeEmbeddings

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import SystemMessage
from langchain.prompts import HumanMessagePromptTemplate

# permet de passer un paramètre entre les différents composants d'une chaîne
from langchain_core.runnables import RunnablePassthrough

from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser

### ![](../../img/parametrage-ico.png) Paramétrages


In [ ]:
# Gestion credentials VertexAI

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "../../settings/credentials.json"
google.auth.default()

In [6]:
# Étape 1 : Charger le document PDF avec PyMuPDFLoader
pdf_path = "./TechWave.pdf"
loader = PyPDFLoader(pdf_path)
documents = loader.load()

print("Number of extracted documents : ", len(documents))

could not convert string to float: '0.00-10' : FloatObject (b'0.00-10') invalid; use 0.0 instead
could not convert string to float: '0.00-10' : FloatObject (b'0.00-10') invalid; use 0.0 instead
could not convert string to float: '0.00-10' : FloatObject (b'0.00-10') invalid; use 0.0 instead
could not convert string to float: '0.00-10' : FloatObject (b'0.00-10') invalid; use 0.0 instead


Number of extracted documents :  36


In [7]:
# Étape 2 : Diviser le document en morceaux plus petits
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
split_documents = text_splitter.split_documents(documents)

print("Number of splitted documents : ", len(split_documents))

Number of splitted documents :  93


In [8]:
embeddings = FakeEmbeddings(size=4096)

# Étape 3 : Stocker les documents divisés dans une ChromaDB
vectorstore = Chroma.from_documents(split_documents, embedding=embeddings)

In [9]:
# Étape 4 : Définir le modèle de réponse avec Pydantic
class Response(BaseModel):
    answer: str = Field(description="Answer to the question")


# Étape 5 : Définir le JsonOutputParser
parser = JsonOutputParser(pydantic_object=Response)

# Étape 6 : Définir le LLM
llm = VertexAI()

In [10]:
# Étape 7 : Définir le ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages(
    [
        # Message système pour spécifier le comportement
        SystemMessage(content=("You are a helpful assistant.")),
        # Message système précisant le format de sortie
        SystemMessage(content=(parser.get_format_instructions())),
        # Message humain à partir d'un template
        HumanMessagePromptTemplate.from_template(
            """Answer the question based only on the following context:
        {context}

        Question: {question}
        """
        ),
    ]
)

In [11]:
# Étape 8 : Utiliser ChromaDB comme retriever
retriever = vectorstore.as_retriever()

# Étape 9 : Chaîner les éléments
retrieval_chain = (
    {"context": retriever, "question": RunnablePassthrough()} |
    prompt |
    llm |
    parser
)

### ![](../../img/jouer-ico.png) Exécution

In [13]:
# Exemple d'utilisation
question = "Quelle entreprise a rédigé ce document ?"
response = retrieval_chain.invoke({"question": question})

print(response)

{'answer': 'Le document a été rédigé par WEnvision et SFEIR.'}
